<a href="https://colab.research.google.com/github/Shankar-Hadimani/ai-ml-engineering/blob/master/notebook/tf_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Oct 16 20:50:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from __future__ import  absolute_import, division, print_function, unicode_literals
try:
  !pip uninstall  tb-nightly tensorboardX tensorboard
  !pip install tb-nightly
except Exception:
  pass

import tensorflow as tf
import os
import datetime
import tensorflow_datasets as tfds

%load_ext tensorboard


Uninstalling tensorboard-2.3.0:
  Would remove:
    /usr/local/bin/tensorboard
    /usr/local/lib/python3.6/dist-packages/tensorboard-2.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorboard/*
Proceed (y/n)? y
  Successfully uninstalled tensorboard-2.3.0
     |████████████████████████████████| 10.6MB 5.6MB/s 


In [3]:
import pkg_resources
for entry_point in pkg_resources.iter_entry_points('tensorboard_plugins'):
  print(entry_point.dist)


tensorboard 2.3.0
tensorboard-plugin-wit 1.7.0


In [4]:
print(tf.__version__)

2.3.0


In [5]:
!ls -alrt /usr/local/lib/python3.6/dist-packages/tensorboard*

/usr/local/lib/python3.6/dist-packages/tensorboard_plugin_wit-1.7.0.dist-info:
total 44
-rw-r--r-- 1 root staff   92 Oct 14 16:37 WHEEL
-rw-r--r-- 1 root staff   23 Oct 14 16:37 top_level.txt
-rw-r--r-- 1 root staff  214 Oct 14 16:37 METADATA
-rw-r--r-- 1 root staff   93 Oct 14 16:37 entry_points.txt
-rw-r--r-- 1 root staff 8386 Oct 14 16:42 RECORD
-rw-r--r-- 1 root staff    4 Oct 14 16:42 INSTALLER
drwxr-sr-x 2 root staff 4096 Oct 14 16:42 .
drwxrwsr-x 1 root staff 4096 Oct 16 20:58 ..

/usr/local/lib/python3.6/dist-packages/tensorboard_plugin_wit:
total 52
-rw-r--r-- 1 root staff 19137 Oct 14 16:37 wit_plugin.py
-rw-r--r-- 1 root staff  2934 Oct 14 16:37 wit_plugin_loader.py
-rw-r--r-- 1 root staff     0 Oct 14 16:37 __init__.py
drwxr-sr-x 4 root staff  4096 Oct 14 16:42 _vendor
drwxr-sr-x 3 root staff  4096 Oct 14 16:42 _utils
drwxr-sr-x 2 root staff  4096 Oct 14 16:42 static
drwxr-sr-x 2 root staff  4096 Oct 14 16:42 __pycache__
drwxr-sr-x 6 root staff  4096 Oct 14 16:42 .
drwxrwsr

In [6]:
!rm -r /usr/local/lib/python3.6/dist-packages/tensorboardcolab-0.0.22.dist-info

In [7]:
print(tf.__version__)

2.3.0


In [8]:
tf.config.experimental.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:XLA_CPU:0', device_type='XLA_CPU'),
 LogicalDevice(name='/device:XLA_GPU:0', device_type='XLA_GPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [10]:
dataset, info = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00', with_info=True)
train_dataset = dataset['train']

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0.incompleteKX5BTW/amazon_us_reviews-train.tfrecord


ERROR:absl:Statistics generation doesn't work for nested structures yet



Dataset amazon_us_reviews downloaded and prepared to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0. Subsequent calls will reuse this data.


In [11]:
info

tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

In [12]:
print(train_dataset)

<DatasetV1Adapter shapes: {data: {customer_id: (), helpful_votes: (), marketplace: (), product_category: (), product_id: (), product_parent: (), product_title: (), review_body: (), review_date: (), review_headline: (), review_id: (), star_rating: (), total_votes: (), verified_purchase: (), vine: ()}}, types: {data: {customer_id: tf.string, helpful_votes: tf.int32, marketplace: tf.string, product_category: tf.string, product_id: tf.string, product_parent: tf.string, product_title: tf.string, review_body: tf.string, review_date: tf.string, review_headline: tf.string, review_id: tf.string, star_rating: tf.int32, total_votes: tf.int32, verified_purchase: tf.int64, vine: tf.int64}}>


In [14]:
train_dataset.output_shapes

Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.


Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.


{'data': {'customer_id': TensorShape([]),
  'helpful_votes': TensorShape([]),
  'marketplace': TensorShape([]),
  'product_category': TensorShape([]),
  'product_id': TensorShape([]),
  'product_parent': TensorShape([]),
  'product_title': TensorShape([]),
  'review_body': TensorShape([]),
  'review_date': TensorShape([]),
  'review_headline': TensorShape([]),
  'review_id': TensorShape([]),
  'star_rating': TensorShape([]),
  'total_votes': TensorShape([]),
  'verified_purchase': TensorShape([]),
  'vine': TensorShape([])}}

In [15]:
len(list(train_dataset))

104975

In [16]:
BUFFER_SIZE = 30000
BATCH_SIZE = 128

In [17]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE, reshuffle_each_iteration=False)

In [19]:
# print the each record, each features is an tensor object unlike pandas dataframe.
# so to view sample  data, iterate over picked records
for reviews in train_dataset.take(2):  
  print(reviews)   


{'data': {'customer_id': <tf.Tensor: shape=(), dtype=string, numpy=b'18902918'>, 'helpful_votes': <tf.Tensor: shape=(), dtype=int32, numpy=2>, 'marketplace': <tf.Tensor: shape=(), dtype=string, numpy=b'US'>, 'product_category': <tf.Tensor: shape=(), dtype=string, numpy=b'Mobile_Electronics'>, 'product_id': <tf.Tensor: shape=(), dtype=string, numpy=b'B008FDTICA'>, 'product_parent': <tf.Tensor: shape=(), dtype=string, numpy=b'210401663'>, 'product_title': <tf.Tensor: shape=(), dtype=string, numpy=b'iKross 3.5mm white and Pink Rabbit Portable Mini Stereo Speaker for iphone 6, Blackberry, iPhone, iPod, iPad, Smartphone, Tablets, Cell Phone'>, 'review_body': <tf.Tensor: shape=(), dtype=string, numpy=b'my sister like much that why I give up 5 stars. I write for make everybody buy one thank u very much.'>, 'review_date': <tf.Tensor: shape=(), dtype=string, numpy=b'2013-01-01'>, 'review_headline': <tf.Tensor: shape=(), dtype=string, numpy=b'really nice'>, 'review_id': <tf.Tensor: shape=(), dty

In [27]:
# view selective features from 10 records from tensor dataset.
for reviews in train_dataset.take(10):
  review_text = reviews['data']
  print(review_text.get('review_body').numpy())
  print(review_text.get('star_rating'))
  print(tf.where(review_text.get('star_rating')>3,1,0).numpy())

b'my sister like much that why I give up 5 stars. I write for make everybody buy one thank u very much.'
tf.Tensor(5, shape=(), dtype=int32)
1
b"I'm favorably impressed.  Really impressed.<br /><br />The Bluetooth pairing was instantaneous, and I was up and running in less than a minute.<br /><br />The fit and form of this product is beautiful.  I'd have expected to pay twice as much.<br /><br />Super satisfied."
tf.Tensor(5, shape=(), dtype=int32)
1
b'Like them so well I bought more'
tf.Tensor(5, shape=(), dtype=int32)
1
b'Love the receiver, however mine is having issues with connection problems after i removed the face the first time and since then i have not removed it, but the reoccurring issue with it wanting to not display anything on the display or randomly change the coloring from blue to red is becoming rather annoying. I will have to give a call out to sony and see what would be the cause before my factory warranty ends.'
tf.Tensor(4, shape=(), dtype=int32)
1
b'Good Stuff'
tf